In [56]:
from scipy.stats import poisson
import numpy as np

lead_time = 1 / 12

lambdas = np.array([6, 1, 0.2, 0.1, 0.05])
costs = np.array([1000, 5000, 8000, 20000, 100000])  
n = len(lambdas)

# calculate mu
mus = lambdas * lead_time

budget = 200000

# Hilfsfunktion1
def calculate_aggregated_beta(S):
    betas = poisson.cdf(S, mus)
    aggregated_beta = np.sum(lambdas * betas) / np.sum(lambdas)
    return aggregated_beta

# hilfsfunction2
def calculate_failure_prob(S):
    return 1 - calculate_aggregated_beta(S)

def calculate_item_failure_prob(S, j):
    return 1 - poisson.cdf(S[j], mus[j])

def calculate_system_failure_prob(S):
    item_failures = 1 - poisson.cdf(S, mus)
    return item_failures


# Greedy
#initial stock level = 0
S = np.zeros(n, dtype=int)

# initial costs
total_costs = 0

current_failure = calculate_failure_prob(S)

i = 0
while True:
    i += 1
    delta = np.zeros(n)
    for j in range(n):

        # Budget-Check
        if np.sum(S * costs) + costs[j] > budget:
            delta[j] = -np.inf
            continue

        failure_j = 1 - poisson.cdf(S[j], mus[j])   
        failure_j_plus = 1 - poisson.cdf(S[j] + 1, mus[j])

        delta[j] = (failure_j - failure_j_plus ) /costs[j]



    j_star = np.argmax(delta)

    print(f"\nIteration {i}")
    print(f"delta = {delta*1000}")
    print(f"best: j = {j_star+1}, delta = {delta[j_star]}")

    S[j_star] += 1
    total_costs = np.sum(S * costs)

    if total_costs > budget:
        S[j_star] -= 1
        break

    system_avail = np.sum((1- poisson.cdf(S, mus))
    ) 

    print(f"S = {S}, System avail = {system_avail}, costs = {total_costs}")




Iteration 1
delta = [3.03265330e-01 1.53340736e-02 2.04889886e-03 4.13208872e-04
 4.14934167e-05]
best: j = 1, delta = 0.00030326532985631673
S = [1 0 0 0 0], System avail = 0.19914484749612316, costs = 1000

Iteration 2
delta = [7.58163325e-02 1.53340736e-02 2.04889886e-03 4.13208872e-04
 4.14934167e-05]
best: j = 1, delta = 7.581633246407916e-05
S = [2 0 0 0 0], System avail = 0.123328515032044, costs = 2000

Iteration 3
delta = [1.26360554e-02 1.53340736e-02 2.04889886e-03 4.13208872e-04
 4.14934167e-05]
best: j = 2, delta = 1.533407357715537e-05
S = [2 1 0 0 0], System avail = 0.04665814714626715, costs = 7000

Iteration 4
delta = [1.26360554e-02 6.38919732e-04 2.04889886e-03 4.13208872e-04
 4.14934167e-05]
best: j = 1, delta = 1.2636055410679914e-05
S = [3 1 0 0 0], System avail = 0.034022091735587234, costs = 8000

Iteration 5
delta = [1.57950693e-03 6.38919732e-04 2.04889886e-03 4.13208872e-04
 4.14934167e-05]
best: j = 3, delta = 2.048898862128376e-06
S = [3 1 1 0 0], System a